In [2]:
import os
from PIL import Image
import torchvision.transforms as transforms
import torch
import numpy as np

from src.datamodule.datamodule import ImageForgeryDatamMdule
from timm.models.swin_transformer import SwinTransformer
from timm.models.layers import SelectAdaptivePool2d
import timm
from torch import nn
import torch_dct as dct
from src.model.cnn_gru import HybridCNNGRU

In [3]:
# state_dict = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True).state_dict()
# model = SwinTransformer(        
#             drop_rate=0.9,
#             proj_drop_rate=0.9,
#             attn_drop_rate=0.9,
#             drop_path_rate=0.9,
#         )
# model.load_state_dict(state_dict)
# model.head = nn.Identity()
# model(image).shape

In [4]:
image = torch.rand(1, 3, 224, 224)

In [5]:
HybridCNNGRU(d_model=256, hidden_size=512, input_channels=3)(image).shape

torch.Size([1, 225, 256])